# H2O Tutorial: EEG Eye State Classification


Author: Erin LeDell

Contact: erin@h2o.ai

This tutorial steps through a quick introduction to H2O's Python API. The goal of this tutorial is to introduce through a complete example H2O's capabilities from Python. 

Most of the functionality for a [Pandas](http://pandas.pydata.org/) `DataFrame` is exactly the same syntax for an `H2OFrame`, so if you are comfortable with Pandas, data frame manipulation will come naturally to you in H2O.  The modeling syntax in the H2O Python API may also remind you of [scikit-learn](http://scikit-learn.org/stable/).  

References: [H2O Python API documentation](http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Pydoc.html) and [H2O general documentation](http://docs.h2o.ai)

## Install H2O in Python

### Prerequisites

This tutorial assumes you have Python 2.7 installed.  The `h2o` Python package has a few dependencies which can be installed using [pip](http://pip.readthedocs.org/en/stable/installing/). The packages that are required are (which also have their own dependencies):
```bash
pip install requests
pip install tabulate
pip install scikit-learn 
```
If you have any problems (for example, installing the `scikit-learn` package), check out [this page](https://github.com/h2oai/h2o-3/blob/master/h2o-docs/src/product/howto/FAQ.md#python) for tips.

### Install h2o

Once the dependencies are installed, you can install H2O.  We will use the [latest stable](http://h2o-release.s3.amazonaws.com/h2o/latest_stable.html) version of the `h2o` package, which is currently "Tibshirani-8."  The installation instructions are on the "Install in Python" tab on [this page](http://h2o-release.s3.amazonaws.com/h2o/rel-tibshirani/8/index.html).

```bash
# The following command removes the H2O module for Python (if it already exists).
pip uninstall h2o

# Next, use pip to install this version of the H2O Python module.
pip install http://h2o-release.s3.amazonaws.com/h2o/rel-tibshirani/8/Python/h2o-3.6.0.8-py2.py3-none-any.whl
```

For reference, the Python documentation for the latest stable release of H2O is [here](http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Pydoc.html).

## Start up an H2O cluster

In a Python terminal, we can import the `h2o` package and start up an H2O cluster.

In [1]:
import h2o

# Start an H2O Cluster on your local machine
h2o.init()

H2O cluster uptime:,2 hours 28 minutes 19 seconds 651 milliseconds
H2O cluster version:,3.6.0.3
H2O cluster name:,H2O_started_from_python
H2O cluster total nodes:,1
H2O cluster total memory:,3.56 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


If you already have an H2O cluster running that you'd like to connect to (for example, in a multi-node Hadoop environment), then you can specify the IP and port of that cluster as follows:

In [114]:
# This will not actually do anything since it's a fake IP address
# h2o.init(ip="123.45.67.89", port=54321)

## Download EEG Data

The following code downloads a copy of the [EEG Eye State](http://archive.ics.uci.edu/ml/datasets/EEG+Eye+State#) dataset.  All data is from one continuous EEG measurement with the [Emotiv EEG Neuroheadset](https://emotiv.com/epoc.php). The duration of the measurement was 117 seconds. The eye state was detected via a camera during the EEG measurement and added later manually to the file after analysing the video frames. '1' indicates the eye-closed and '0' the eye-open state. All values are in chronological order with the first measured value at the top of the data.

![Emotiv Headset](http://dissociatedpress.com/wp-content/uploads/2013/03/emotiv-490.jpg)

We can import the data directly into H2O using the `import_file` method in the Python API.  The import path can be a URL, a local path, a path to an HDFS file, or a file on Amazon S3.

In [2]:
csv_url = "http://www.stat.berkeley.edu/~ledell/data/eeg_eyestate_splits.csv"
data = h2o.import_file(csv_url)


Parse Progress: [##################################################] 100%


## Explore Data
Once we have loaded the data, let's take a quick look.  First the dimension of the frame:

In [3]:
data.shape


(14980, 16)

Now let's take a look at the top of the frame:

In [4]:
data.head()

AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,eyeDetection,split
4329.23,4009.23,4289.23,4148.21,4350.26,4586.15,4096.92,4641.03,4222.05,4238.46,4211.28,4280.51,4635.9,4393.85,0,valid
4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.1,0,test
4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,0,train
4328.72,4011.79,4296.41,4155.9,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41,0,train
4326.15,4011.79,4292.31,4151.28,4347.69,4586.67,4095.9,4627.69,4210.77,4244.1,4212.82,4288.21,4632.82,4398.46,0,train
4321.03,4004.62,4284.1,4153.33,4345.64,4587.18,4093.33,4616.92,4202.56,4232.82,4209.74,4281.03,4628.21,4389.74,0,train
4319.49,4001.03,4280.51,4151.79,4343.59,4584.62,4089.74,4615.9,4212.31,4226.67,4201.03,4269.74,4625.13,4378.46,0,test
4325.64,4006.67,4278.46,4143.08,4344.1,4583.08,4087.18,4614.87,4205.64,4230.26,4195.9,4266.67,4622.05,4380.51,0,test
4326.15,4010.77,4276.41,4139.49,4345.13,4584.1,4091.28,4608.21,4187.69,4229.74,4202.05,4273.85,4627.18,4389.74,0,test
4326.15,4011.28,4276.92,4142.05,4344.1,4582.56,4092.82,4608.72,4194.36,4228.72,4212.82,4277.95,4637.44,4393.33,0,train


The first 14 columns are numeric values that represent EEG measurements from the headset.  The "eyeDetection" column is the response.  There is an additional column called "split" that was added (by me) in order to specify partitions of the data (so we can easily benchmark against other tools outside of H2O using the same splits).  I randomly divided the dataset into three partitions: train (60%), valid (%20) and test (20%) and marked which split each row belongs to in the "split" column.

Let's take a look at the column names.  The data contains derived features from the medical images of the tumors.

In [5]:
data.columns

[u'AF3',
 u'F7',
 u'F3',
 u'FC5',
 u'T7',
 u'P7',
 u'O1',
 u'O2',
 u'P8',
 u'T8',
 u'FC6',
 u'F4',
 u'F8',
 u'AF4',
 u'eyeDetection',
 u'split']

To select a subset of the columns to look at, typical Pandas indexing applies:

In [6]:
columns = ['AF3', 'eyeDetection', 'split']
data[columns].head()

AF3,eyeDetection,split
4329.23,0,valid
4324.62,0,test
4327.69,0,train
4328.72,0,train
4326.15,0,train
4321.03,0,train
4319.49,0,test
4325.64,0,test
4326.15,0,test
4326.15,0,train


Now let's select a single column, for example -- the response column, and look at the data more closely:

In [7]:
y = 'eyeDetection'
data[y]

eyeDetection
0
0
0
0
0
0
0
0
0
0


It looks like a binary response, but let's validate that assumption:

In [8]:
data[y].unique()

C1
0
1


If you don't specify the column types when you import the file, H2O makes a guess at what your column types are.  If there are 0's and 1's in a column, H2O will automatically parse that as numeric by default.  

Therefore, we should convert the response column to a more efficient "enum" representation -- in this case it is a categorial variable with two levels, 0 and 1.  If the only column in my data that is categorical is the response, I typically don't bother specifying the column type during the parse, and instead use this one-liner to convert it aftewards:

In [9]:
data[y] = data[y].asfactor()

Now we can check that there are two levels in our response column:

In [10]:
data[y].nlevels()

[2]

We can query the categorical "levels" as well ('0' and '1' stand for "eye open" and "eye closed") to see what they are:

In [11]:
data[y].levels()

[['0', '1']]

We may want to check if there are any missing values, so let's look for NAs in our dataset.  For tree-based methods like GBM and RF, H2O handles missing feature values automatically, so it's not a problem if we are missing certain feature values.  However, it is always a good idea to check to make sure that you are not missing any of the training labels.  

To figure out which, if any, values are missing, we can use the `isna` method on the diagnosis column.  The columns in an H2O Frame are also H2O Frames themselves, so all the methods that apply to a Frame also apply to a single column.

In [12]:
data.isna()

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
data[y].isna()

C1
0
0
0
0
0
0
0
0
0
0


The `isna` method doesn't directly answer the question, "Does the response column contain any NAs?", rather it returns a 0 if that cell is not missing (Is NA? FALSE == 0) and a 1 if it is missing (Is NA? TRUE == 1).  So if there are no missing values, then summing over the whole column should produce a summand equal to 0.0.  Let's take a look:

In [14]:
data[y].isna().sum()

0.0

Great, no missing labels.  :-)

Out of curiosity, let's see if there is any missing data in this frame:

In [15]:
data.isna().sum()

0.0

The sum is still zero, so there are no missing values in any of the cells.

The next thing I may wonder about in a binary classification problem is the distribution of the response in the training data.  Is one of the two outcomes under-represented in the training set?  Many real datasets have what's called an "imbalanace" problem, where one of the classes has far fewer training examples than the other class.  Let's take a look at the distribution:

In [16]:
data[y].table()

eyeDetection,Count
0,8257
1,6723


Ok, the data is not exactly evenly distributed between the two classes -- there are more 0's than 1's in the dataset.  However, this level of imbalance shouldn't be much of an issue for the machine learning algos.  (We will revisit this later in the modeling section below).

Let's calculate the percentage that each class represents:

In [17]:
n = data.shape[0]  # Total number of training samples
data[y].table()['Count']/n

Count
0.551202
0.448798


### Split H2O Frame into a train and test set

So far we have explored the original dataset (all rows).  For the machine learning portion of this tutorial, we will break the dataset into three parts: a training set, validation set and a test set.

If you want H2O to do the splitting for you, you can use the `split_frame` method.  However, we have explicit splits that we want (for reproducibility reasons), so we can just subset the Frame to get the partitions we want.  

Subset the `data` H2O Frame on the "split" column:

In [18]:
train = data[data['split']=="train"]
train.shape

(8988, 16)

In [19]:
valid = data[data['split']=="valid"]
valid.shape

(2996, 16)

In [20]:
test = data[data['split']=="test"]
test.shape

(2996, 16)

## Machine Learning in H2O

We will do a quick demo of the H2O software using a Gradient Boosting Machine (GBM).  The goal of this problem is to train a model to predict eye state (open vs closed) from EEG data. 

### Train and Test a GBM model

In [21]:
# Import H2O GBM:
from h2o.estimators.gbm import H2OGradientBoostingEstimator


We first create a `model` object of class, `"H2OGradientBoostingEstimator"`.  This does not actually do any training, it just sets the model up for training by specifying model parameters.

In [22]:
model = H2OGradientBoostingEstimator(distribution='bernoulli',
                                    ntrees=100,
                                    max_depth=4,
                                    learn_rate=0.1)

#### Specify the predictor set and response

The `model` object, like all H2O estimator objects, has a `train` method, which will actually perform model training.  At this step we specify the training and (optionally) a validation set, along with the response and predictor variables.

The `x` argument should be a list of predictor names in the training frame, and `y` specifies the response column.  We have already set `y = "eyeDetector"` above, but we still need to specify `x`.

In [23]:
x = list(train.columns)
x

[u'AF3',
 u'F7',
 u'F3',
 u'FC5',
 u'T7',
 u'P7',
 u'O1',
 u'O2',
 u'P8',
 u'T8',
 u'FC6',
 u'F4',
 u'F8',
 u'AF4',
 u'eyeDetection',
 u'split']

In [29]:
del x[12:14]  #Remove the 13th and 14th columns, 'eyeDetection' and 'split'
x

[u'AF3',
 u'F7',
 u'F3',
 u'FC5',
 u'T7',
 u'P7',
 u'O1',
 u'O2',
 u'P8',
 u'T8',
 u'FC6',
 u'F4']

Now that we have specified `x` and `y`, we can train the model:

In [30]:
model.train(x=x, y=y, training_frame=train, validation_frame=valid)


gbm Model Build Progress: [##################################################] 100%


### Inspect Model

The type of results shown when you print a model, are determined by the following:
- Model class of the estimator (e.g. GBM, RF, GLM, DL)
- The type of machine learning problem (e.g. binary classification, multiclass classification, regression)
- The data you specify (e.g. `training_frame` only, `training_frame` and `validation_frame`, or `training_frame` and `nfolds`)

Below, we see a GBM Model Summary, as well as training and validation metrics since we supplied a `validation_frame`.  Since this a binary classification task, we are shown the relevant performance metrics, which inclues: MSE, R^2, LogLoss, AUC and Gini.  Also, we are shown a Confusion Matrix, where the threshold for classification is chosen automatically (by H2O) as the threshold which maximizes the F1 score.

The scoring history is also printed, which shows the performance metrics over some increment such as "number of trees" in the case of GBM and RF.

Lastly, for tree-based methods (GBM and RF), we also print variable importance.

In [142]:
print model

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1448515953177_2

Model Summary:


,number_of_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,100.0,23613.0,4.0,4.0,4.0,10.0,16.0,14.9




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.114026790434
R^2: 0.539835211
LogLoss: 0.376005292812
AUC: 0.936370388939
Gini: 0.872740777878

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.43076103173:


,0,1,Error,Rate
0,4102.0,814.0,0.1656,(814.0/4916.0)
1,534.0,3538.0,0.1311,(534.0/4072.0)
Total,4636.0,4352.0,0.15,(1348.0/8988.0)



Maximum Metrics: Maximum metrics at their respective thresholds



...

metric,threshold,value,idx
max f1,0.5,0.8,211.0
max f2,0.3,0.9,269.0
max f0point5,0.6,0.8,158.0
max accuracy,0.5,0.8,205.0
max precision,1.0,1.0,0.0
max absolute_MCC,0.5,0.7,210.0
max min_per_class_accuracy,0.5,0.8,211.0



Scoring History:


,timestamp,duration,number_of_trees,training_MSE,training_logloss,training_AUC,training_classification_error,validation_MSE,validation_logloss,validation_AUC,validation_classification_error
,2015-11-25 21:36:34,0.171 sec,1.0,0.2,0.7,0.7,0.3,0.2,0.7,0.7,0.3
,2015-11-25 21:36:34,0.232 sec,2.0,0.2,0.7,0.7,0.3,0.2,0.7,0.7,0.3
,2015-11-25 21:36:34,0.268 sec,3.0,0.2,0.6,0.8,0.3,0.2,0.6,0.8,0.3
,2015-11-25 21:36:34,0.302 sec,4.0,0.2,0.6,0.8,0.3,0.2,0.6,0.8,0.3
,2015-11-25 21:36:34,0.360 sec,5.0,0.2,0.6,0.8,0.3,0.2,0.6,0.8,0.3
---,---,---,---,---,---,---,---,---,---,---,---
,2015-11-25 21:36:38,3.852 sec,77.0,0.1,0.4,0.9,0.2,0.1,0.4,0.9,0.2
,2015-11-25 21:36:38,3.917 sec,78.0,0.1,0.4,0.9,0.2,0.1,0.4,0.9,0.2
,2015-11-25 21:36:38,3.978 sec,79.0,0.1,0.4,0.9,0.2,0.1,0.4,0.9,0.2
,2015-11-25 21:36:38,4.043 sec,80.0,0.1,0.4,0.9,0.2,0.1,0.4,0.9,0.2



Variable Importances:


variable,relative_importance,scaled_importance,percentage
P7,1308.6,1.0,0.2
O1,1043.4,0.8,0.2
F7,835.9,0.6,0.1
AF3,730.2,0.6,0.1
F4,465.7,0.4,0.1
O2,465.4,0.4,0.1
T8,340.7,0.3,0.1
FC6,282.6,0.2,0.0
FC5,249.7,0.2,0.0
F3,239.0,0.2,0.0


### Model Performance on a Test Set

Once a model has been trained, you can also use it to make predictions on a test set.  In the case above, we just ran the model once, so our validation set (passed as `validation_frame`), could have also served as a "test set."  We technically have already created test set predictions and evaluated test set performance.  

However, when performing model selection over a variety of model parameters, it is common for users to train a variety of models (using different parameters) using the training set, `train`, and a validation set, `valid`.  Once the user selects the best model (based on validation set performance), the true test of model performance is performed by making a final set of predictions on the held-out (never been used before) test set, `test`.

You can use the `model_performance` method to generate predictions on a new dataset.  The results are stored in an object of class, `"H2OBinomialModelMetrics"`.  

In [31]:
perf = model.model_performance(test)
print perf.__class__

<class 'h2o.model.metrics_base.H2OBinomialModelMetrics'>


Individual model performance metrics can be extracted using methods like `r2`, `auc` and `mse`.  In the case of binary classification, we may be most interested in evaluating test set Area Under the ROC Curve (AUC).  

In [32]:
perf.r2()

0.49537936872725086

In [33]:
perf.auc()

0.91671733144306

In [34]:
perf.mse()

0.12372290870105287

### Cross-validated Performance

To perform k-fold cross-validation, you use the same code as above, but you specify `nfolds` as an integer greater than 1, or add a "fold_column" to your H2O Frame which indicates a fold ID for each row.

Unless you have a specific reason to manually assign the observations to folds, you will find it easiest to simply use the `nfolds` argument.

When performing cross-validation, you can still pass a `validation_frame`, but you can also choose to use the original dataset that contains all the rows.  We will cross-validate a model below using the original H2O Frame which is called `data`.

In [35]:
cvmodel = H2OGradientBoostingEstimator(distribution='bernoulli',
                                       ntrees=100,
                                       max_depth=4,
                                       learn_rate=0.1,
                                       nfolds=5)

cvmodel.train(x=x, y=y, training_frame=data)



gbm Model Build Progress: [##################################################] 100%


This time around, we will simply pull the training and cross-validation metrics out of the model.  To do so, you use the `auc` method again, and you can specify `train` or `xval` as `True` to get the correct metric.

In [36]:
print cvmodel.auc(train=True)
print cvmodel.auc(xval=True)

0.92657469777
0.909384518188


### Grid Search

One way of evaluting models with different parameters is to perform a grid search over a set of parameter values.  For example, in GBM, here are three model parameters that may be useful to search over:
- `ntrees`: Number of trees
- `max_depth`: Maximum depth of a tree
- `learn_rate`: Learning rate in the GBM

We will define a grid as follows:

In [37]:
ntrees_opt = [5,50,100]
max_depth_opt = [2,3,5]
learn_rate_opt = [0.1,0.2]

hyper_params = {'ntrees': ntrees_opt, 
                'max_depth': max_depth_opt,
                'learn_rate': learn_rate_opt}

Define an `"H2OGridSearch"` object by specifying the algorithm (GBM) and the hyper parameters:

In [38]:
from h2o.grid.grid_search import H2OGridSearch

gs = H2OGridSearch(H2OGradientBoostingEstimator, hyper_params = hyper_params)

An `"H2OGridSearch"` object also has a `train` method, which is used to train all the models in the grid.

In [39]:
gs.train(x=x, y=y, training_frame=train, validation_frame=valid)


gbm Grid Build Progress: [##################################################] 100%


### Compare Models

In [40]:
print gs


Grid Search Results for H2OGradientBoostingEstimator:


Model Id,"Hyperparameters: [learn_rate, ntrees, max_depth]",mse
Grid_GBM_py_17_model_python_1448515953177_10_model_17,"[0.2, 100, 5]",0.1
Grid_GBM_py_17_model_python_1448515953177_10_model_16,"[0.2, 50, 5]",0.1
Grid_GBM_py_17_model_python_1448515953177_10_model_8,"[0.1, 100, 5]",0.1
Grid_GBM_py_17_model_python_1448515953177_10_model_7,"[0.1, 50, 5]",0.1
Grid_GBM_py_17_model_python_1448515953177_10_model_14,"[0.2, 100, 3]",0.1
Grid_GBM_py_17_model_python_1448515953177_10_model_13,"[0.2, 50, 3]",0.1
Grid_GBM_py_17_model_python_1448515953177_10_model_5,"[0.1, 100, 3]",0.1
Grid_GBM_py_17_model_python_1448515953177_10_model_4,"[0.1, 50, 3]",0.2
Grid_GBM_py_17_model_python_1448515953177_10_model_11,"[0.2, 100, 2]",0.2
Grid_GBM_py_17_model_python_1448515953177_10_model_15,"[0.2, 5, 5]",0.2


In [41]:
# print out the auc for all of the models
for g in gs:
    print g.model_id + " auc: " + str(g.auc())

Grid_GBM_py_17_model_python_1448515953177_10_model_6 auc: 0.832367242163
Grid_GBM_py_17_model_python_1448515953177_10_model_9 auc: 0.694192742594
Grid_GBM_py_17_model_python_1448515953177_10_model_13 auc: 0.891897083178
Grid_GBM_py_17_model_python_1448515953177_10_model_15 auc: 0.853560718899
Grid_GBM_py_17_model_python_1448515953177_10_model_16 auc: 0.969346889232
Grid_GBM_py_17_model_python_1448515953177_10_model_17 auc: 0.98940421078
Grid_GBM_py_17_model_python_1448515953177_10_model_1 auc: 0.760300329424
Grid_GBM_py_17_model_python_1448515953177_10_model_10 auc: 0.794593697697
Grid_GBM_py_17_model_python_1448515953177_10_model_3 auc: 0.729356404691
Grid_GBM_py_17_model_python_1448515953177_10_model_4 auc: 0.854010340319
Grid_GBM_py_17_model_python_1448515953177_10_model_11 auc: 0.82936795932
Grid_GBM_py_17_model_python_1448515953177_10_model_2 auc: 0.793885558323
Grid_GBM_py_17_model_python_1448515953177_10_model_12 auc: 0.76833561695
Grid_GBM_py_17_model_python_1448515953177_10_mo

The "best" model in terms of validation set AUC is "Grid_GBM_py_17_model_python_1448515953177_10_model_17" with an AUC of 0.994422431426. 

In [42]:
best_model = h2o.get_model("Grid_GBM_py_17_model_python_1448515953177_10_model_17")
best_model.auc()

0.9894042107804035

The last thing we may want to do is generate predictions on the test set using the "best" model, and evaluate the test set AUC.

In [43]:
best_perf = best_model.model_performance(test)
best_perf.auc()

0.9609710824540837

The test set AUC is approximately 0.96.  Not bad!!